## Generate SteamID

In [ ]:
# find steamIDs
def steamid_to_64bit(steamid):
    steam64id = 76561197960265728
    
    id_split = steamid.split(":")
    steam64id += int(id_split[2]) * 2
    if id_split[1] == "1":
        steam64id += 1
    return steam64id

# simple format of steam id: STEAM_1:1:66138017 --> we generate the last 8 digits consecutively
Id_list = []
for num in range(20000000, 20000100):
    steam_Id = "STEAM_1:1:" + str(num)
    Id_list.append(steam_Id)

# generate 64-bit steamID list and csv file 
steamId_64bit = []
with open("steamid.csv", "w") as f:
    for i in Id_list:
        steamId_64bit.append(str(steamid_to_64bit(i)))
        steamId = str(steamid_to_64bit(i)) + ",\n"
        f.write(steamId)

# Input SteamIDs and get user country from GetPlayerSummaries

In [ ]:
import numpy as np
import json
import requests

In [ ]:
# Step 1: open steam id from csv file, multi_id is a list to store the ids
with open("steamid.csv", "r") as f:
    multi_id = f.read().split(",")


id_pub = []  # Step 3: ids in multi_id may not be available, we need to filter out valid ids and put into id_pub list
def GetPlayerSummaries(multi_id):
    with open("id_location.csv", "w") as f:   # Step 4: we want to save user locations in csv file
    
        # Step 5:
        # url format: https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/?key=<API key>&format=json&steamids=<steamID_1>,<steamID_2>,<steamID_3>
        # that means we need to add ',' between the ids
        steamId = ""
        for i in multi_id:
            if steamId != "":
                steamId = steamId + "," + i
            else:
                steamId = i
        
        params = {
            "steamids" : steamId
        }
    
        url = requests.get("https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/?key=81C83AF57A45C03A06CA67C939151C18&format=json", headers=headers, params=params)    
        d = json.loads(url.text)
        
        # Step 6:
        # if the id from multi_id is valid:
        # 1. get country information    2. add the id and country info into id_pub list for record/ later use --- format: id,country for the cooresponding id
        for i in range(len(multi_id) - 1):
                        
            try:             
                if d['response']['players'][i]['loccountrycode']:
                    x = multi_id[i] + "," + d['response']['players'][i]['loccountrycode'] + "\n"
                    id_pub.append(x)                                      
            except:
                d['response']['players'][i]['loccountrycode'] = np.nan      
        
        # Step 7: Finally, write id_pub list into id_location.csv
        for i in range(len(id_pub)):
            f.write(id_pub[i])

# Step 2: We get 100 ids from multi_id list each time, and input to 'GetPlayerSummaries' function. We set a for loop and input the ids for 500 times.
for i in range(0, 500):
    GetPlayerSummaries(multi_id[(0+100*i):(100+100*i)])      ## input --- multi_id[0:100], multi_id[100:200], multi_id[200:300], ...
# --> after that, we can get total 500*100 ids